In [ ]:
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

In [ ]:
%pip install  \
    "langchain>=0.0.350" \
    "transformers>=4.24,<5" \
    sqlalchemy -U \
    "faiss-cpu>=1.7,<2" \
    "pypdf>=3.8,<4" \
    pinecone-client==2.2.4 \
    apache-beam==2.52. \
    tiktoken==0.5.2 \
    "ipywidgets>=7,<8" \
    matplotlib==3.8.2 \
    anthropic==0.9.0

In [ ]:
%pip install pydantic==1.10.8

RESTART THE KERNEL

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

SET UP MODEL

In [ ]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v2",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


# Prompt
In the context of large language models (LLMs), a prompt refers to the initial input provided to the model to generate a desired response. This prompt serves as a guide or instruction for the model, influencing the direction and content of its generated text. The quality and specificity of the prompt significantly impact the relevance and coherence of the model's output. By crafting effective prompts, users can steer LLMs towards producing desired responses that align with their objectives or requirements.

## Prompt in langchain

In langchain framework has been defined a specific class to define and manage prompts :
class langchain_core.prompts.prompt.PromptTemplate

Prompt templates are predefined recipes for generating prompts for language models.
A template may include instructions, few-shot examples, and specific context and questions appropriate for a given task.

```python
#Instantiation using from_template (recommended)
prompt = PromptTemplate.from_template("Say {foo}")
prompt.format(foo="bar")

#Instantiation using initializer
prompt = PromptTemplate(input_variables=["foo"], template="Say {foo}")
prompt.format(foo="bar")
```

Here is an example of how to use a PromptTemplate

In [16]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

# Prompting Techniques
Prompting techniques involve designing prompts in a way that optimizes the performance of large language models (LLMs) for specific tasks.

## Zero-shot prompt templates
Zero-shot prompt engineering refers to the practice of designing prompts for language models in such a way that the model can perform a task without explicit training examples or fine-tuning on that specific task

Large LLMs today, such as GPT-3.5, are tuned to follow instructions and are trained on large amounts of data; so they are capable of performing some tasks "zero-shot."

Here is one example :

In [15]:
prompt_template = PromptTemplate.from_template(
    "Classify the text into neutral, negative or positive.\n" 
    "Text: I think the vacation is okay.\n"
    "Sentiment:"
)

response = textgen_llm(prompt_template.template)


print_ww(response)

Note that in the prompt above we didn't provide the model with any examples of text alongside their classifications, the LLM already understands "sentiment" -- that's the zero-shot capabilities at work.

### Exercise 1 :



## Few-shot prompt templates
A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses the word whatpu is:
We were traveling in Africa and we saw these very cute whatpus.
 
To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses the word farduddle is:

## COT : Chain of thoughts

## Zero-shot COT : 